In [ ]:
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies

!pip3 install wget
!apt-get install sox libsndfile1 ffmpeg
!pip3 install unidecode


## Install NeMo
!python -m pip install nemo_toolkit[all]

# !pip install torchtext==0.10.1
# !pip install matplotlib==3.1.3
"""
Remember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!
Alternatively, you can uncomment the exit() below to crash and restart the kernel, in the case
that you want to use the "Run All Cells" (or similar) option.
"""
exit()

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=e486006ba29a47e53bb544bf493b1e69f73396db80460e6d2ec6c7eaf3404555
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-4ubuntu0.18.04.2).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
The following additional packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3
Suggested packages:
  file libsox-fmt-all
The following NEW packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3 sox
0 upgraded, 8 newly installed, 0 to remove and 37 not upgraded.
Need to get 760 kB of archives.
After this operation, 6,717 kB of additional disk s

In [ ]:
# Папка, где будет размещаться датасет Golos
data_dir = '/content'

In [ ]:
 #!rm -rf test

In [ ]:
! ls /content

sample_data  test  test.tar  train  train_crowd9.tar


In [ ]:
import glob
import os
import subprocess
import tarfile
import wget

# Загрузка датасета Golos
def load(golos_url, fname):
  if not os.path.exists(os.path.join(data_dir, fname)):
      golos_path = wget.download(golos_url, os.path.join(data_dir, fname))
      print(f"Dataset downloaded at: {golos_path}")
  else:
      print("Tarfile already exists.")
      golos_path = os.path.join(data_dir, fname)
  return golos_path

golos_path = load("https://sc.link/XKk", "train_crowd9.tar")

if not os.path.exists(os.path.join(data_dir, '/train/')):
    tar = tarfile.open(golos_path)
    tar.extractall(path=data_dir)

golos_path = load("https://sc.link/Kqr", "test.tar")

if not os.path.exists(os.path.join(data_dir, '/test/')):
    tar = tarfile.open(golos_path)
    tar.extractall(path=data_dir)

In [ ]:
if not os.path.exists(os.path.join(data_dir, '/test/')):
    tar = tarfile.open('/content/test.tarr')
    tar.extractall(path=data_dir)

In [ ]:
import json
import pandas as pd

In [ ]:
! head train/manifest.jsonl

{"id": "77b380796d242cf5bc09cfb551cffecd", "audio_filepath": "crowd/0/77b380796d242cf5bc09cfb551cffecd.wav", "text": "\u0430\u043b\u0438\u0431\u0438", "duration": 1.21}
{"id": "404e1eecbc38a687e036739f2e5dd6d3", "audio_filepath": "crowd/0/404e1eecbc38a687e036739f2e5dd6d3.wav", "text": "\u0441\u043a\u0430\u0437\u043a\u0430", "duration": 2.9}
{"id": "f913db484595c1bcdd9476194b08a1f4", "audio_filepath": "crowd/0/f913db484595c1bcdd9476194b08a1f4.wav", "text": "\u043d\u0430\u0439\u0434\u0438 \u0432 \u043d\u0435\u0442\u0435 \u043c\u0443\u043b\u044c\u0442 \u0441\u0442\u0430\u043b\u044c\u043d\u043e\u0439 \u0433\u0438\u0433\u0430\u043d\u0442", "duration": 4.76}
{"id": "eca448acf1712fa7ca08aede3d2df1ac", "audio_filepath": "crowd/0/eca448acf1712fa7ca08aede3d2df1ac.wav", "text": "\u043c\u043e\u043d\u0430 \u043b\u0438\u0437\u0430", "duration": 2.3}
{"id": "a366eecb0a85ef76463e1f4b5188b698", "audio_filepath": "crowd/0/a366eecb0a85ef76463e1f4b5188b698.wav", "text": "\u0441\u043c\u043e\u0442\u0440\u04

In [ ]:
from tqdm import tqdm

In [ ]:
data = []

In [ ]:
with open("train/manifest.jsonl", 'r', encoding='utf8') as f:
    for line in tqdm(f):
        item = json.loads(line)
        num_chars = len(item['text'])
        orig = item['text'].split()
        num_words = len(orig)
        word_rate = round(num_words / item['duration'], 2)
        char_rate = round(num_chars / item['duration'], 2)
        if (item["audio_filepath"].startswith('crowd/9') and item['duration']>0):
          data.append(
                {
                    'audio_filepath': item['audio_filepath'],
                    'duration': round(item['duration'], 2),
                    'num_words': num_words,
                    'num_chars': num_chars,
                    'word_rate': word_rate,
                    'char_rate': char_rate,
                    'text': item['text'],
                }
              )

1103799it [00:14, 78581.85it/s]


In [ ]:
data

Теперь в папке `/content` должна размещаться папка `test` с тестовыми данных датасета Golos. Она содержит файлы манифесты `./test/crowd/manifest.jsonl`, `./test/farfield/manifest.jsonl` и аудио файлы в подпапках `./test/crowd/files/` и `./test/farfield/files/`.

Мы можем загрузить и посмотреть данные. Например файл `./test/crowd/files/e632f7d39c15e7edfc665b91e6f2071f.wav` это четырехсекундная запись мужчины, который произносит фразу `Купить моющее средство`. Чтобы убедиться в этом давайте послушаем запись:

In [ ]:
import librosa
import IPython.display as ipd

# Load and listen to the audio file
example_file = '/content/train/crowd/9/00001d3fd5d2c8cd00882afe632a6b0c.wav'

audio, sample_rate = librosa.load(example_file)

ipd.Audio(example_file, rate=sample_rate)

В задаче распознавания речи, если этот WAV файлявляется нашим входом в систему, то фраза `Купить моющее средство` будет желаемым выходом из системы.

Давайте изобразим форму волны по точкам, которые содержатся в файлах. 
Этот формат отображения аудио, который скорее всего вам знаком, така как используется во многих аудио радакторах и плеерах:

## Сверточная модель распознавания речи
### Jasper модель

Мы будем тренировать небольшую модель [Jasper (Just Another SPeech Recognizer) model](https://arxiv.org/abs/1904.03288) с нуля (инициализируется случайным образом).
Архитектуры Jasper состоят из повторяющейся блочной структуры, которая использует 1D-свертки.
В модели Jasper_KxR подблоки `R` (состоящие из 1D свертки, batchnorm, ReLU и dropout) группируются в один блок, который затем повторяется `K` раз.
Также есть один дополнительный блок в начале и еще несколько в конце, которые инвариантны к `K` и `R`, идалее используем CTC loss.

### Модель QuartzNet

QuartzNet является улучшенным вариантом Jasper с ключевым отличием в том, что она использует 1D-свертки, разделяемые по временному каналу. Это позволяет ему значительно сократить количество весов при сохранении аналогичной точности.

Модели Jasper/QuartzNet выглядят так (на фото изображена модель QuartzNet):

![QuartzNet with CTC](https://developer.nvidia.com/blog/wp-content/uploads/2020/05/quartznet-model-architecture-1-625x742.png)

# Использование NeMo для распонавания речи

Теперь мы знаем что такое задача автоматичского распознавания речи и речевые данные, давайте использовать NeMo для распознавания речи.

Мы будем использовать **Neural Modules (NeMo) toolkit**, поэтому нужно скачать и установить все ее зависимости. Для этого следуйте инструкции в репозитории [GitHub page](https://github.com/NVIDIA/NeMo), или документации [documentation](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/).

NeMo позволяет нам легко использовать все необходимые компоненты для нашей модели: dataloader,  промежуточные сверточные или рекурентные слои, разные loss  функции без необходимости разбираться в деталях реализации разных моедлей. В NeMo содержатся готовые реализованные модели в которых достаточно подать свои данные и задать гиперпараметры для обучения.

In [ ]:
# NeMo's "core" package
import nemo
# NeMo's ASR collection - this collections contains complete ASR models and
# building blocks (modules) for ASR
import nemo.collections.asr as nemo_asr

[NeMo W 2021-11-30 09:07:44 optimizers:50] Apex was not found. Using the lamb or fused_adam optimizer will error out.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



## Исползование предобученной модели


Коллекция для распознавания речи в NeMo содержит готовые блоки, которые можно использовать чтобы тренировать и использовать свою модель. Кроме этого существет ряд предобученных моделей, которые можно просто скачать и исползовать. Давайте скачаем и инициализируем готовую модель QuartzNet15x5, обученную на открытом датасете Golos.



In [ ]:
# asr_model = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_ru_quartznet15x5")

load("https://sc.link/ZMv", "QuartzNet15x5_golos.nemo")
asr_model = nemo_asr.models.EncDecCTCModel.restore_from(os.path.join(data_dir, "QuartzNet15x5_golos.nemo"))

Dataset downloaded at: /content/QuartzNet15x5_golos.nemo


[NeMo W 2021-11-30 09:07:55 modelPT:131] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: train/golos_and_mcv.jsonl
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я
    batch_size: 64
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    num_workers: 20
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    parser: ru
    
[NeMo W 2021-11-30 09:07:55 modelPT:138] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a

[NeMo I 2021-11-30 09:07:55 features:265] PADDING: 16
[NeMo I 2021-11-30 09:07:55 features:282] STFT using torch
[NeMo I 2021-11-30 09:08:14 save_restore_connector:149] Model EncDecCTCModel was successfully restored from /content/QuartzNet15x5_golos.nemo.


Теперь указваем список фалов которые мы хотим транскрибировать и передаем в нашу модель. Это будет работать для относительно коротких аудио (<25 секунд) файлов. 

In [ ]:
files = [data_dir + "/train/crowd/9/00001d3fd5d2c8cd00882afe632a6b0c.wav"]
transcriptions = asr_model.transcribe(paths2audio_files=files)

for fname, transcription in zip(files, transcriptions):
  print(f"Audio in {fname} was recognized as: {transcription}")

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

[NeMo W 2021-11-30 09:08:14 patch_utils:50] torch.stft() signature has been updated for PyTorch 1.7+
    Please update PyTorch to remain compatible with later versions of NeMo.
[NeMo W 2021-11-30 09:08:14 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
      ) // self.conv.stride[0] + 1
    


Audio in /content/train/crowd/9/00001d3fd5d2c8cd00882afe632a6b0c.wav was recognized as: новой ладоге


In [ ]:
files = []
for d in data:
  full_path = data_dir + '/train/' + d['audio_filepath']
  files.append(full_path)
transcriptions = asr_model.transcribe(paths2audio_files=files)

Transcribing:   0%|          | 0/19949 [00:00<?, ?it/s]

[NeMo W 2021-11-30 09:08:22 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
      ) // self.conv.stride[0] + 1
    


In [ ]:
!pip install python-Levenshtein
from Levenshtein import distance as levenshtein_distance

     |████████████████████████████████| 50 kB 3.1 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149868 sha256=3ccbdc352ac41915abbee61d33f809695a1e4f47f5bc66d0a214c89c3072cb7d
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [ ]:
list_with_distances = []
big_dist = []
for fname, transcription in tqdm(zip(files, transcriptions)):
  for d in data:
    full_path = data_dir + "/train/"+ d['audio_filepath']
    if full_path == fname:
      n = levenshtein_distance(transcription, d["text"])
      list_with_distances.append([fname, n])
      if n > 1:
        big_dist.append(fname)

79796it [39:10, 33.95it/s]


In [ ]:
with open ('/content/dist.txt', 'a') as f:
  for each in big_dist:
    f.write(each[0] +'\n')


In [ ]:
 with open('/content/transcriptions.txt', 'a') as f: 
  for fname, transcription in tqdm(zip(files, transcriptions)):
    f.write(fname +', ' + transcription)

79796it [00:00, 668107.96it/s]


Это было просто! Но существует масса сценариев, когда мы хотим дообучить модель на наших данных или обучить заново. Например это модель не сможет распознать украинский язык и скоре всего будет плохо распознавать речь в телефонных разговорах. Поэтому, если вы применяете модель на своих данных, то без сомнения, вам нужно дообучать модель на них!

## Обучение 


Для обучения нужно подготовить данные в нужном формате. Для этого добавим абсолютные пути к относительным в наших манифестах и используем их для обучения.

In [ ]:
dt = []
with open("train/manifest.jsonl", 'r', encoding='utf8') as f:
    for line in tqdm(f):
        item = json.loads(line)
        if item['audio_filepath'].startswith('crowd/9'):
          dt.append(item)
with open("train/manifest_9.jsonl", 'a', encoding='utf8') as f:
  for d in dt:
    d_str = str(d)
    d_str = d_str.replace("'", '"')
    f.write(d_str + '\n')


1103799it [00:09, 111901.94it/s]


In [41]:
new_man = []
path = '/content/train/'
with open("train/manifest_9.jsonl", 'r', encoding='utf8') as f:
    for line in tqdm(f):
      item = json.loads(line)
      if os.path.join(path, item["audio_filepath"]) in big_dist:
        new_man.append(item)

with open("train/new_manifest.jsonl", 'a', encoding='utf8') as f:
    for line in new_man:
      l = str(line)
      l = l.replace("'", '"')
      f.write(str(l) + '\n')

79796it [00:00, 120497.59it/s]


In [44]:

# --- Building Manifest Files --- #
import json

# Function to build a manifest
def build_manifest(manifest_rel, manifest_abs):
    manifest_path = os.path.split(os.path.abspath(manifest_rel))[0]
    with open(manifest_rel, 'r') as fin:
        with open(manifest_abs, 'w') as fout:
            for line in fin:
                metadata = json.loads(line)
                metadata["audio_filepath"]=os.path.join(manifest_path, metadata["audio_filepath"])
                json.dump(metadata, fout)
                fout.write('\n')
                
# Building Manifests
print("******")
train_rel = os.path.join(data_dir, 'test/crowd/manifest.jsonl')
train_abs = os.path.join(data_dir, 'test/crowd/crowd_test.jsonl') # файл с абсолютными путями
if not os.path.isfile(train_abs):
  build_manifest(train_rel, train_abs)
test_manifest = train_abs
print("test_manifest", test_manifest)

train_rel = os.path.join(data_dir, 'train/new_manifest.jsonl')
train_abs = os.path.join(data_dir, 'train/train_new.jsonl')
if not os.path.isfile(train_abs):
  build_manifest(train_rel, train_abs)
train_manifest = train_abs
print("train_manifest", train_manifest)

******
test_manifest /content/test/crowd/crowd_test.jsonl
train_manifest /content/train/train_new.jsonl


In [45]:
# ! head /content/test/crowd/crowd_test.jsonl
! head /content/train/train_new.jsonl

{"id": "470053ba0c2f5367a8517de16e37ab80", "audio_filepath": "/content/train/crowd/9/470053ba0c2f5367a8517de16e37ab80.wav", "text": "\u043a\u044b\u0437\u044b\u043b\u043e\u043c", "duration": 2.62}
{"id": "fcb95d798b16661d0562fe7f310adffb", "audio_filepath": "/content/train/crowd/9/fcb95d798b16661d0562fe7f310adffb.wav", "text": "\u043c\u043e\u0436\u0438 \u0434\u0430\u0441 \u043a\u0440\u0443\u0437\u0438\u0441", "duration": 3.0}
{"id": "678a1b8f27d6605beaff54871c601401", "audio_filepath": "/content/train/crowd/9/678a1b8f27d6605beaff54871c601401.wav", "text": "\u0442\u0435\u0442\u0435", "duration": 2.24}
{"id": "050413911a4ec5a14fab40fa1d0a3fdd", "audio_filepath": "/content/train/crowd/9/050413911a4ec5a14fab40fa1d0a3fdd.wav", "text": "\u0430\u0431\u0430\u043a\u0430\u043d", "duration": 2.4}
{"id": "69d602731f52c9b5729ff5f33fe9f299", "audio_filepath": "/content/train/crowd/9/69d602731f52c9b5729ff5f33fe9f299.wav", "text": "\u043b\u0438\u043d\u044c\u0445\u0430\u0439", "duration": 2.28}
{"id": "

### Задаем модель при помощи YAML конфиг файла


Ниже приведен конфиг файл, давайте рассмотрим его и найдем части описанной архитектуры Jasper. Модель (model) содержит поле под названием `encoder` с под-полем `jasper` который состоит из списка полей. Каждое поле в списке задает конфигурацию блока в нашей моделе. Каждый блок выглядит примерно так:

```
- filters: 128
  repeat: 1
  kernel: [11]
  stride: [2]
  dilation: [1]
  dropout: 0.2
  residual: false
  separable: true
  se: true
  se_context_size: -1
```


Первый элемент в спике соответствует первому блоку в Jasper архитектуре.

Параметры обучающего и тестового датасета в полях (`train_ds`) и (`validation_ds`)

Конфиг в формате YAML позволяем легко и в читаемой форме читать и модифицировать модель без необходимости менять код пррограммы.

### Использование PyTorch Lightning

NeMo модели и модули могуть исползоваться в любом PyTorch проекте где ожидается тип torch.nn.Module.

Однако, NeMo модели созданы на основе [PytorchLightning's](https://github.com/PyTorchLightning/pytorch-lightning) LightningModule, поэтому рекомендуется использовать PytorchLightning для обучения и дообучения (fine-tuning) так как это позволяет легко применять mixed precision и распределенное обучение. Давайте создадим объект Trainer для обучения на GPU 5 эпох.

In [58]:
import pytorch_lightning as pl

In [71]:
params_model = asr_model._cfg

# --- Config Information ---#
# try:
#     from ruamel.yaml import YAML
# except ModuleNotFoundError:
#     from ruamel_yaml import YAML
# config_path = '/usr/local/lib/python3.7/dist-packages/nemo/collections/asr/models/configs/'

# yaml = YAML(typ='safe')
# with open(config_path) as f:
#     params = yaml.load(f)
# print(params)
params_model['train_ds']['parser']='base'
params_model['validation_ds']['parser']='base'
params_model['test_ds']['parser']='base'
params_model['train_ds']['manifest_filepath'] = train_manifest
params_model['train_ds']['batch_size']=16
params_model['validation_ds']['manifest_filepath'] = test_manifest
params_model['validation_ds']['batch_size']=16
params_model['test_ds']['manifest_filepath'] = test_manifest
params_model['test_ds']['batch_size']=16

In [ ]:
params_model

{'sample_rate': 16000, 'repeat': 5, 'dropout': 0.0, 'separable': True, 'labels': [' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я'], 'optim': {'name': 'novograd', 'lr': 0.001, 'betas': [0.9, 0.98], 'weight_decay': 0.001, 'sched': {'name': 'CosineAnnealing', 'warmup_steps': 500, 'warmup_ratio': None, 'min_lr': 0.0, 'last_epoch': -1}}, 'train_ds': {'manifest_filepath': '/content/train/crowd/crowd_train.jsonl', 'sample_rate': 16000, 'labels': [' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я'], 'batch_size': 12, 'trim_silence': False, 'max_duration': 20.0, 'min_duration': 0.1, 'num_workers': 20, 'shuffle': True, 'is_tarred': False, 'tarred_audio_filepaths': None, 'tarred_shard_strategy': 'scatter', 'parser': 'base'}, 'validation_ds': {'manifest_filepath': '/conten

In [60]:
params_model['optim']['lr'] = 0.001

Затем мы создадим экземпляр и модель ASR на основе нашего файла ``config.yaml`` из предыдущего раздела. 
Обратите внимание, что это этап, на котором мы также сообщаем модели, где находятся наши обучающие и тестовые манифесты.

In [70]:
from omegaconf import DictConfig
trainer = pl.Trainer(gpus=1, max_epochs=1, precision=16)
asr_model = nemo_asr.models.EncDecCTCModel(cfg=DictConfig(params_model), trainer=trainer)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


[NeMo I 2021-11-30 12:38:15 collections:173] Dataset loaded with 21 files totalling 0.02 hours
[NeMo I 2021-11-30 12:38:15 collections:174] 1 files were filtered totalling 0.01 hours


[NeMo W 2021-11-30 12:38:15 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


[NeMo I 2021-11-30 12:38:15 collections:173] Dataset loaded with 9994 files totalling 11.20 hours
[NeMo I 2021-11-30 12:38:15 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2021-11-30 12:38:15 collections:173] Dataset loaded with 9994 files totalling 11.20 hours
[NeMo I 2021-11-30 12:38:15 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2021-11-30 12:38:15 features:265] PADDING: 16
[NeMo I 2021-11-30 12:38:15 features:282] STFT using torch


In [68]:
import torch
torch.cuda.empty_cache()

In [72]:
asr_model.setup_training_data(train_data_config=params_model['train_ds'])
asr_model.setup_test_data(test_data_config=params_model['test_ds'])
asr_model.setup_validation_data(val_data_config=params_model['validation_ds'])

[NeMo I 2021-11-30 12:38:36 collections:173] Dataset loaded with 21 files totalling 0.02 hours
[NeMo I 2021-11-30 12:38:36 collections:174] 1 files were filtered totalling 0.01 hours


[NeMo W 2021-11-30 12:38:36 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


[NeMo I 2021-11-30 12:38:36 collections:173] Dataset loaded with 9994 files totalling 11.20 hours
[NeMo I 2021-11-30 12:38:36 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2021-11-30 12:38:36 collections:173] Dataset loaded with 9994 files totalling 11.20 hours
[NeMo I 2021-11-30 12:38:36 collections:174] 0 files were filtered totalling 0.00 hours


Мы можем начать обучение всего одной строчкой!

In [73]:
# Start training!!!
trainer.fit(asr_model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[NeMo W 2021-11-30 12:38:56 modelPT:471] The lightning trainer received accelerator: <pytorch_lightning.accelerators.gpu.GPUAccelerator object at 0x7f136b545290>. We recommend to use 'ddp' instead.


[NeMo I 2021-11-30 12:38:56 modelPT:561] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.001
        weight_decay: 0.001
    )
[NeMo I 2021-11-30 12:38:56 lr_scheduler:752] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7f1373082850>" 
    will be used during training (effective maximum steps = 2) - 
    Parameters : 
    (warmup_steps: 500
    warmup_ratio: null
    min_lr: 0.0
    last_epoch: -1
    max_steps: 2
    )



  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18.9 M
2 | decoder           | ConvASRDecoder                    | 34.9 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     
------------------------------------------------------------------------
18.9 M    Trainable params
0         Non-trainable params
18.9 M    Total params
37.859    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

[NeMo W 2021-11-30 12:38:56 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    
[NeMo W 2021-11-30 12:38:58 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b

Training: 0it [00:00, ?it/s]

[NeMo W 2021-11-30 12:39:05 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
      "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
    


Validating: 0it [00:00, ?it/s]

There we go! We've put together a full training pipeline for the model and trained it for 5 epochs.

If you'd like to save this model checkpoint for loading later (e.g. for fine-tuning, or for continuing training), you can simply call `first_asr_model.save_to(<checkpoint_path>)`. Then, to restore your weights, you can rebuild the model using the config (let's say you call it `first_asr_model_continued` this time) and call `first_asr_model_continued.restore_from(<checkpoint_path>)`.

### After Training: Monitoring Progress and Changing Hyperparameters
We can  now start Tensorboard to see how training went. Recall that WER stands for Word Error Rate and so the lower it is, the better.

In [ ]:
try:
  from google import colab
  COLAB_ENV = True
except (ImportError, ModuleNotFoundError):
  COLAB_ENV = False

# Load the TensorBoard notebook extension
if COLAB_ENV:
  %load_ext tensorboard
  %tensorboard --logdir lightning_logs/
else:
  print("To use tensorboard, please use this notebook in a Google Colab environment.")

We could improve this model by playing with hyperparameters. We can look at the current hyperparameters with the following:

In [ ]:
print(params_model['optim'])

Let's say we wanted to change the learning rate. To do so, we can create a `new_opt` dict and set our desired learning rate, then call `<model>.setup_optimization()` with the new optimization parameters.

In [74]:
import copy
new_opt = copy.deepcopy(params_model['optim'])
new_opt['lr'] = 0.01
asr_model.setup_optimization(optim_config=DictConfig(new_opt))
# And then you can invoke trainer.fit(first_asr_model)

[NeMo W 2021-11-30 12:47:10 modelPT:471] The lightning trainer received accelerator: <pytorch_lightning.accelerators.gpu.GPUAccelerator object at 0x7f136b545290>. We recommend to use 'ddp' instead.


[NeMo I 2021-11-30 12:47:10 modelPT:561] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.01
        weight_decay: 0.001
    )
[NeMo I 2021-11-30 12:47:10 lr_scheduler:752] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7f1373436c10>" 
    will be used during training (effective maximum steps = 2) - 
    Parameters : 
    (warmup_steps: 500
    warmup_ratio: null
    min_lr: 0.0
    last_epoch: -1
    max_steps: 2
    )


(Novograd (
 Parameter Group 0
     amsgrad: False
     betas: [0.9, 0.98]
     eps: 1e-08
     grad_averaging: False
     initial_lr: 0.01
     lr: 1.996007984031936e-05
     weight_decay: 0.001
 ),
 {'frequency': 1,
  'interval': 'step',
  'monitor': 'loss',
  'reduce_on_plateau': False,
  'scheduler': <nemo.core.optim.lr_scheduler.CosineAnnealing at 0x7f1373436c10>})

In [75]:
# Point to the data we'll use for fine-tuning as the training set
asr_model.setup_training_data(train_data_config=params_model['train_ds'])

# Point to the new validation data for fine-tuning
asr_model.setup_validation_data(val_data_config=params_model['validation_ds'])

# And now we can create a PyTorch Lightning trainer and call `fit` again.
trainer = pl.Trainer(gpus=1, max_epochs=1)
trainer.fit(asr_model)

[NeMo I 2021-11-30 12:47:16 collections:173] Dataset loaded with 21 files totalling 0.02 hours
[NeMo I 2021-11-30 12:47:16 collections:174] 1 files were filtered totalling 0.01 hours


[NeMo W 2021-11-30 12:47:16 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


[NeMo I 2021-11-30 12:47:16 collections:173] Dataset loaded with 9994 files totalling 11.20 hours
[NeMo I 2021-11-30 12:47:16 collections:174] 0 files were filtered totalling 0.00 hours


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[NeMo W 2021-11-30 12:47:16 modelPT:471] The lightning trainer received accelerator: <pytorch_lightning.accelerators.gpu.GPUAccelerator object at 0x7f136b545290>. We recommend to use 'ddp' instead.


[NeMo I 2021-11-30 12:47:16 modelPT:561] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.01
        weight_decay: 0.001
    )
[NeMo I 2021-11-30 12:47:16 lr_scheduler:752] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7f1373113210>" 
    will be used during training (effective maximum steps = 2) - 
    Parameters : 
    (warmup_steps: 500
    warmup_ratio: null
    min_lr: 0.0
    last_epoch: -1
    max_steps: 2
    )



  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18.9 M
2 | decoder           | ConvASRDecoder                    | 34.9 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     
------------------------------------------------------------------------
18.9 M    Trainable params
0         Non-trainable params
18.9 M    Total params
75.718    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

[NeMo W 2021-11-30 12:47:19 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
      ) // self.conv.stride[0] + 1
    
[NeMo W 2021-11-30 12:47:20 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:408: UserWarning: The number of training samples (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
      f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"
   

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

## Inference

Let's have a quick look at how one could run inference with NeMo's ASR model.

First, ``EncDecCTCModel`` and its subclasses contain a handy ``transcribe`` method which can be used to simply obtain audio files' transcriptions. It also has batch_size argument to improve performance.

In [ ]:
print(asr_model.transcribe(paths2audio_files=files,
                                 batch_size=4))

Below is an example of a simple inference loop in pure PyTorch. It also shows how one can compute Word Error Rate (WER) metric between predictions and references.

In [76]:
# Bigger batch-size = bigger throughput
# params['model']['validation_ds']['batch_size'] = 16
# Setup the test data loader and make sure the model is on GPU
# first_asr_model.setup_test_data(test_data_config=params['model']['validation_ds'])
asr_model.cuda()

# We will be computing Word Error Rate (WER) metric between our hypothesis and predictions.
# WER is computed as numerator/denominator.
# We'll gather all the test batches' numerators and denominators.
wer_nums = []
wer_denoms = []

# Loop over all test batches.
# Iterating over the model's `test_dataloader` will give us:
# (audio_signal, audio_signal_length, transcript_tokens, transcript_length)
# See the AudioToCharDataset for more details.
for test_batch in asr_model.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]        
        log_probs, encoded_len, greedy_predictions = asr_model(
            input_signal=test_batch[0], input_signal_length=test_batch[1]
        )
        # Notice the model has a helper object to compute WER
        asr_model._wer.update(greedy_predictions, targets, targets_lengths)
        _, wer_num, wer_denom = asr_model._wer.compute()
        asr_model._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions

# We need to sum all numerators and denominators first. Then divide.
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

[NeMo W 2021-11-30 12:55:16 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
      ) // self.conv.stride[0] + 1
    


RuntimeError: ignored